## The compatibility of sQUlearn and scikit-learn

This notebook demonstrates the ability of sQUlearn to smoothly interact with scikit-learn routines. We first define a simple toy function and use the Chebychev quantum feature map to define a fidelity quantum kernel. The trainable parameters of the quantum feature map are sampled randomly. The aim in mind is to solve a regression problem using QSVR. Subsequently, we apply scikit-learn's cross validation score. Finally we demonstrate how to optimize the inherent hyperparameters of the SVR pipeline ($C$ and $\epsilon$) using GridSearchCV.

In [2]:
import numpy as np

from squlearn import Executor
from squlearn.feature_map import ChebPQC
from squlearn.kernel.matrix import FidelityKernel
from squlearn.kernel.ml import QSVR

In [3]:
# Generate data set (analogously to qgpr_workflow.ipynb example)
def func(x):
    return np.sin(3 * x) * x * x


# define training and test data
x = np.linspace(-1, 1, 100)
y = func(x)
x_train = np.linspace(-1, 1, 30)
y_train = func(x_train)

In [4]:
# We can define an qml method of choice and use additional scikit-learnfunctions
# to validate the model

# 1. Step: define QML method
# set up feature map with trainable parameters being randomly initialized
fmap = ChebPQC(num_qubits=4, num_features=1, num_layers=2)

# sample variational parameters of feature map randomly and pass them to kernel definition
np.random.seed(0)
param = np.random.rand(fmap.num_parameters)
q_kernel_fqk = FidelityKernel(
    feature_map=fmap, executor=Executor("statevector_simulator"), initial_parameters=param
)

# define QSVR instance with previously defined kernel
qsvr_fqk = QSVR(quantum_kernel=q_kernel_fqk, C=1e-3)

In [5]:
# 2. Step load and use additional scikit-learn routines
from sklearn.model_selection import cross_val_score, cross_val_predict

print("Cross validation score:", cross_val_score(qsvr_fqk, x_train.reshape(-1, 1), y_train, cv=5))
print(
    "Cross validation prediction:",
    cross_val_predict(qsvr_fqk, x_train.reshape(-1, 1), y_train, cv=5),
)

Cross validation score: [-18.12670951  -6.44089286   0.31412181  -6.43812685 -18.12626799]
Cross validation prediction: [ 0.09453931  0.09296693  0.09228168  0.09181882  0.09151399  0.09134121
  0.08982308  0.09024509  0.09072591  0.09125676  0.09182972  0.09243742
 -0.0024385  -0.00151414 -0.00056691  0.00039215  0.00135188  0.00230087
 -0.09251079 -0.09187206 -0.0912602  -0.09068376 -0.09015211 -0.08967597
 -0.09126054 -0.09143937 -0.09176706 -0.09227573 -0.09304126 -0.09481676]


In [6]:
# we could also use scikit-learnroutines to optimize hyperparameters
from sklearn.model_selection import GridSearchCV

param_grid = {"C": [1e-3, 1e-2, 1e-1, 1, 10], "epsilon": [0.01, 0.1, 0.5]}
qsvr_grid = GridSearchCV(qsvr_fqk, param_grid)

In [7]:
qsvr_grid.fit(x_train.reshape(-1, 1), y_train)

GridSearchCV(estimator=QSVR(C=0.001, cache_size=200, epsilon=0.1, max_iter=-1,
                            quantum_kernel=<squlearn.kernel.matrix.fidelity_kernel.FidelityKernel object at 0x000001A2FF77BDF0>,
                            shrinking=True, tol=0.001, verbose=False),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'epsilon': [0.01, 0.1, 0.5]})

In [8]:
from sklearn.metrics import mean_squared_error as mse

print("Best parameters:", qsvr_grid.best_params_)
print(
    "MSE with optimized hyperparameters:", mse(y_train, qsvr_grid.predict(x_train.reshape(-1, 1)))
)

Best parameters: {'C': 0.01, 'epsilon': 0.01}
MSE with optimized hyperparameters: 0.04248217090511489
